In [1]:
#!pip install psycopg
#!pip install psycopg2


In [2]:
import psycopg2, os

print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="",
    port='',
    dbname="",
    user="",
    password="")
print('Connected to the PostgreSQL database...')

Connecting to the PostgreSQL database...
Connected to the PostgreSQL database...


In [3]:
conn

<connection object at 0x00000231CA8E2DF0; dsn: 'user=postgres password=xxx dbname=APAN5400 host=localhost port=5432', closed: 0>

In [4]:
# create a cursor
cur = conn.cursor()
cur

<cursor object at 0x00000231CA9065E0; closed: 0>

In [5]:
#execute a statement to make sure the cursor works and hence connected to the database

print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 16.2 (Debian 16.2-1.pgdg120+2) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [6]:
#Creating the Neighborhoods table

createCmd = """ CREATE TABLE Neighborhoods (
Id SERIAL PRIMARY KEY,
State VARCHAR(255),
Neighborhood_group VARCHAR(255),
Neighborhood VARCHAR(255)
);
"""

cur.execute(createCmd)
conn.commit()

print("Database table created ...")

Database table created ...


In [7]:
#DELETING TABLE!!!!!!
#cur.execute("DROP TABLE Neighborhoods CASCADE")
#conn.commit()

In [8]:
#cur.execute("DROP TABLE Hosts CASCADE")
#conn.commit()

In [9]:
#Creating Hosts Table

createCmd = """ CREATE TABLE Hosts (
    Id SERIAL PRIMARY KEY,
    Host_url VARCHAR(255),
    Name VARCHAR(255),
    Since DATE,
    Location VARCHAR(255),
    About TEXT,
    Response_time VARCHAR(50),
    Response_rate DOUBLE PRECISION,
    Acceptance_rate DOUBLE PRECISION,
    Is_superhost BOOLEAN,
    Neighborhood VARCHAR(255),
    Listings_count INTEGER,
    Total_listings_count INTEGER,
    Verifications TEXT,
    Has_profile_pic BOOLEAN,
    Identity_verified BOOLEAN
);
"""

cur.execute(createCmd)
conn.commit()

print("Database table created ...")

Database table created ...


In [10]:
#Creating Listings Table

createCmd = """ CREATE TABLE Listings(
    Id BIGINT PRIMARY KEY,
    Name VARCHAR(255),
    Host_id INTEGER REFERENCES Hosts(Id),
    Neighborhood_id INTEGER REFERENCES Neighborhoods(Id),
    Description TEXT,
    Neighborhood_overview TEXT,
    Picture_url VARCHAR(4096),
    Latitude DOUBLE PRECISION,
    Longitude DOUBLE PRECISION,
    Property_type VARCHAR(50),
    Accommodates INTEGER,
    Bathrooms_text TEXT,
    Beds INTEGER,
    Bedrooms INTEGER,
    Amenities VARCHAR(4096),
    Price DOUBLE PRECISION,
    Minimum_nights INTEGER,
    Maximum_nights INTEGER
);
            """

cur.execute(createCmd)
conn.commit()

print("Database table created ...")

Database table created ...


In [11]:
#Creating Reviews Table

createCmd = """CREATE TABLE Reviews (
    Id BIGINT,
    Listing_id BIGINT REFERENCES Listings(Id),
    Date DATE,
    Reviewer_id BIGINT,
    Reviewer_name VARCHAR(255),
    Comments TEXT,
    PRIMARY KEY (Id, Listing_id)
);
            """

cur.execute(createCmd)
conn.commit()

print("Database table created ...")

Database table created ...


In [12]:
#Creating Calendar Table

createCmd = """ CREATE TABLE Calendar (
    Id SERIAL PRIMARY KEY,
    Listing_id BIGINT REFERENCES Listings(Id),
    Date DATE,
    Available BOOLEAN,
    Price DECIMAL(10, 2),
    Adjusted_Price DECIMAL(10, 2)
);

            """

cur.execute(createCmd)
conn.commit()

print("Database table created ...")

Database table created ...


In [13]:
#Processing
import re

def process_rate(rate_str):
    # Check if the rate is "N/A" and return None
    if rate_str == "N/A":
        return None
    
    # If the rate is a percentage, convert to decimal
    try:
        if rate_str.endswith('%'):
            return float(rate_str.strip('%')) / 100
    except ValueError:
        # In case there is an error with the conversion, return None or handle it appropriately
        return None
    
    # If the rate is not a percentage and not "N/A", return it as it is
    # Or you might want to handle this case differently
    return rate_str


def process_boolean(boolean_str):
    
    boolean_str = boolean_str.strip().lower()
    if boolean_str == "":
        return None
    
    if boolean_str == "t":
        return True
    elif boolean_str == "f":
        return False
    else:
        return None
    
def process_datetime(datetime_str):
    
    datetime_str = datetime_str.strip()
    if datetime_str == "":
        return None
    
    else:
        return datetime_str
    
def process_numeric(numeric_str):
    numeric_str = numeric_str.strip()
    if numeric_str == "":
        return None
    try:
        return int(numeric_str)
    except ValueError:
        return None
    


def process_double(value):
    # If the value is None or an empty string, return None
    if value is None or value == '':
        return None
    
    # If the value is already a float, return it as is
    if isinstance(value, float):
        return value

    # If the value is a string, strip it and convert to float
    if isinstance(value, str):
        try:
            return float(value.strip())
        except ValueError:
            # If conversion fails, return None
            return None

    # If the value is some other type, handle appropriately or raise an error
    raise TypeError(f"Value '{value}' is not a string or float and cannot be processed as a double precision number.")

    

def process_currency(currency_str):
    
    if currency_str is None: return None
    try:
        amount = float(re.sub(r"[^\d.]", "", currency_str))
        return amount
    
    except ValueError:
        return None

print("Functions successfully declared ...")

In [14]:
# Path to the CSV file - Loading CSV file - Populating the Neighborhood database
import csv
import psycopg2


def import_neighborhoods_from_csv(csv_file_path, state):
   
    # Open the CSV file
    with open(csv_file_path, newline='') as csvfile:
        # Create a CSV reader object using the file object (csvfile)
        csv_reader = csv.reader(csvfile)
        next(csv_reader)  # Skip the header row
        
        # Prepare the INSERT statement SQL command
        insert_statement = "INSERT INTO Neighborhoods (Neighborhood_group, Neighborhood, State) VALUES (%s, %s, %s)"
        
        # Iterate over the rows in the CSV file
        for row in csv_reader:
            # Extract the neighborhood_group and neighborhood from the current row
            neighborhood_group = row[0]
            neighborhood = row[1]
        
            
            # Execute the INSERT statement
            cur.execute(insert_statement, (neighborhood_group, neighborhood, state))

    # Commit the transaction
    conn.commit()

    
csv_file_path = 'usa-2/New York City/neighbourhoods.csv'
import_neighborhoods_from_csv(csv_file_path, "NY")

csv_file_path2 = 'usa-2/Hawaii/neighbourhoods.csv'
import_neighborhoods_from_csv(csv_file_path2, "HI")

print("Records of neighbourhoods inserted ...")

Records of neighbourhoods inserted ...


In [15]:
#Hosts
import csv
import psycopg2

def fetch_host_ids(cur):
    query = "SELECT id FROM hosts;"
    cur.execute(query)
    results = cur.fetchall()
    return set(row[0] for row in results)


def import_hosts_from_csv(csv_file_path):
  
    
    # Open the CSV file
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        # Create a CSV reader object using the file object (csvfile)
        csv_reader = csv.DictReader(csvfile)
        
        # Prepare the INSERT statement SQL command
        # Make sure the column names match those in your 'CREATE TABLE' statement
        insert_statement = """
        INSERT INTO Hosts (
            Id, Host_url, Name, Since, Location, About,
            Response_time, Response_rate, Acceptance_rate,
            Is_superhost, Neighborhood, Listings_count,
            Total_listings_count, Verifications,
            Has_profile_pic, Identity_verified
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """
        
        # Iterate over the rows in the CSV file
        hosts_by_id = {}
        existing_host_ids = fetch_host_ids(cur)
          
        for row in csv_reader:
        # Use the Id as the key
            key = int(row['host_id'])
            if key in existing_host_ids:
                continue
            if key not in hosts_by_id:
                hosts_by_id[key] = []
            hosts_by_id[key].append(row)
            
        for host_id, rows in hosts_by_id.items():
            # Execute the INSERT statement
            row=rows[0]
            host_acceptance_rate = process_rate(row['host_acceptance_rate'])
            host_response_rate = process_rate(row['host_response_rate'])
            
            cur.execute(insert_statement, (
                row['host_id'], row['host_url'], row['host_name'], process_datetime(row['host_since']),
                row['host_location'], row['host_about'], row['host_response_time'], 
                process_double(host_response_rate), process_double(host_acceptance_rate), process_boolean(row['host_is_superhost']), 
                row['host_neighbourhood'], process_numeric(row['host_listings_count']), process_numeric(row['host_total_listings_count']), 
                row['host_verifications'], process_boolean(row['host_has_profile_pic']), process_boolean(row['host_identity_verified'])
            ))

    # Commit the transaction
    conn.commit()
    

csv_file_path = 'usa-2/New York City/listings_detailed.csv'
import_hosts_from_csv(csv_file_path)

csv_file_path2 = 'usa-2/Hawaii/listings_detailed.csv'
import_hosts_from_csv(csv_file_path2)

print("Records inserted ...")

Records inserted ...


In [16]:
#Listings table 

def fetch_neighborhood_ids(cur):
    query = "SELECT id, neighborhood_group, neighborhood FROM neighborhoods;"
    cur.execute(query)
    results = cur.fetchall()
    
    neighborhood_dict = {}
    for (Id, neighborhood_group, neighborhood) in results:
        # The key is a tuple of (neighborhood_group, neighborhood)
        key = (neighborhood_group, neighborhood)
        neighborhood_dict[key] = Id
        
    return neighborhood_dict



def import_listings_from_csv(csv_file_path):
    
    neighborhood_ids = fetch_neighborhood_ids(cur)
    
    # Open the CSV file
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        # Create a CSV reader object using the file object (csvfile)
        csv_reader = csv.DictReader(csvfile)
        
        # Prepare the INSERT statement SQL command
        insert_statement = """
        INSERT INTO Listings (
            Id, Name, Host_id, Neighborhood_id, Description, Neighborhood_overview, Picture_url,
            Latitude, Longitude, Property_type, Accommodates, Bathrooms_text,
            Beds, Bedrooms, Amenities, Price, Minimum_nights, Maximum_nights
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        for row in csv_reader:
            # Extract the neighborhood_group and neighborhood from the row
            neighborhood_group = row.get('neighbourhood_group_cleansed')
            neighborhood = row.get('neighbourhood_cleansed')
            
            # Use the extracted values to find the corresponding Neighborhood_id
            neighborhood_key = (neighborhood_group, neighborhood)
            neighborhood_id = neighborhood_ids.get(neighborhood_key)
            
            if neighborhood_id is None:
                print(f"Neighborhood ID not found for {neighborhood_key}")
                continue  # Skip this row or handle it appropriately

        
            
            # Here we are assuming the CSV column names match the database field names exactly.
            # You'll need to replace 'column_name' with the actual column names from your CSV.
            cur.execute(insert_statement, (
                row['id'],row['name'], row['host_id'], neighborhood_id, row['description'],
                row['neighborhood_overview'], row['picture_url'], process_double(row['latitude']),
                process_double(row['longitude']), row['property_type'], process_numeric(row['accommodates']),
                row['bathrooms_text'], process_numeric(row['beds']), process_numeric(row['bedrooms']),
                row['amenities'], process_currency(row['price']), process_numeric(row['minimum_nights']), process_numeric(row['maximum_nights'])
            ))

    # Commit the transaction
    conn.commit()

csv_file_path = 'usa-2/New York City/listings_detailed.csv'
import_listings_from_csv(csv_file_path)

csv_file_path2 = 'usa-2/Hawaii/listings_detailed.csv'
import_listings_from_csv(csv_file_path2)

print("Records inserted ...")

Records inserted ...


In [17]:
#Calendar
import csv
import psycopg2

def fetch_listing_ids(cur):
    query = "SELECT id FROM listings;"
    cur.execute(query)
    results = cur.fetchall()
    return set(row[0] for row in results)
        

def import_calendar_from_csv(csv_file_path):
    listing_ids = fetch_listing_ids(cur)
    error_listing_ids = set()
    total = 0
    temp_file_path = 'temp_calendar_data.csv'  # Temporary file for processed data

    # Open the input CSV file and a temporary file for writing processed data
    with open(csv_file_path, newline='') as csvfile, open(temp_file_path, 'w', newline='') as temp_file:
        csv_reader = csv.reader(csvfile)
        writer = csv.writer(temp_file)

        next(csv_reader)  # Skip the header row

        # Iterate over the rows in the input CSV file
        for row in csv_reader:
            total += 1
            listing_id = int(row[0])
            # Check if the listing ID is valid; if not, skip the row
            
            if listing_id not in listing_ids:
                error_listing_ids.add(listing_id)
                continue
            date = row[1]
            available = 'TRUE' if row[2].lower() in ['true', '1', 't'] else 'FALSE'
            price = process_currency(row[3])
            adjusted_price = process_currency(row[4])

            # Write the processed row to the temporary file
            writer.writerow([listing_id, date, available, price, adjusted_price])
    print("processed temp file:", temp_file)
    # Use COPY command to load data from the temporary file into the database
    with open(temp_file_path, 'r') as temp_file:
        cur.copy_expert("COPY calendar (listing_id, date, available, price, adjusted_price) FROM STDIN WITH CSV", temp_file)

    conn.commit()
    
    if len(error_listing_ids) > 0:
        print(f"Warning: {len(error_listing_ids)} listing_ids not found in listing")
        
    print(f"Total rows processed: {total}")

# Usage example
csv_file_path = 'usa-2/New York City/calendar.csv'
import_calendar_from_csv(csv_file_path)

csv_file_path2 = 'usa-2/Hawaii/calendar.csv'
import_calendar_from_csv(csv_file_path2)

print("Data loaded ...")

processed temp file: <_io.TextIOWrapper name='temp_calendar_data.csv' mode='w' encoding='cp1252'>
Total rows processed: 15664741
processed temp file: <_io.TextIOWrapper name='temp_calendar_data.csv' mode='w' encoding='cp1252'>
Total rows processed: 11227079
Data loaded ...


In [18]:
#Reviews_detailed

def import_reviews_from_csv(csv_file_path):
    listing_ids = fetch_listing_ids(cur)
    error_listing_ids = set()
    temp_file_path = 'temp_reviews_data.csv'  # Temporary file for processed data
    
    #Open the input CSV file and a temporary file for writing processed data
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile, open(temp_file_path, 'w', newline='', encoding='utf-8') as temp_file:
        csv_reader = csv.reader(csvfile)
        writer = csv.writer(temp_file)

        next(csv_reader)  # Skip the header row

        
        # Iterate over the rows in the CSV file
        for row in csv_reader:
            # Extract the information from the current row
            listing_id = int(row[0])
            review_id = row[1]
            if listing_id not in listing_ids:
                error_listing_ids.add(listing_id)
                continue
            date = row[2]
            reviewer_id = row[3]
            reviewer_name = row[4]
            comments = row[5]
            
            # Write the processed row to the temporary file
            writer.writerow([review_id, listing_id, date, reviewer_id, reviewer_name, comments])
            
    with open(temp_file_path, 'r', encoding='utf-8') as temp_file:
        cur.copy_expert("COPY reviews (id, listing_id, date, reviewer_id, reviewer_name, comments) FROM STDIN WITH CSV", temp_file)

    
    
    if len(error_listing_ids) > 0:
        print(f"Warning: {len(error_listing_ids)} listing_ids not found in listing")
        
    # Commit the transaction
    conn.commit()
    
csv_file_path = 'usa-2/New York City/reviews_detailed.csv'
import_reviews_from_csv(csv_file_path)

csv_file_path2 = 'usa-2/Hawaii/reviews_detailed.csv'
import_reviews_from_csv(csv_file_path2)

print("Data loaded ...")

Data loaded ...


In [19]:
#IN CASE THERE IS AN ERROR "INSQLTRANSACTION"
conn.rollback()

print("Records have been rolled back ...")

Records have been rolled back


In [21]:
 # Close the cursor and the connection
cur.close()
conn.close()
print("Connection closed ...")

Connection closed
